In [1]:
import networkx as nx
import pickle
import time
import utils
import math

In [2]:
def hyper_gen(level=2, degree=7, VERBOSE=True):
    G = nx.Graph()
    G.add_node(0, x = 0, y = 0, radian_begin = 0, radian_end = 2 * math.pi)
    node_begin = G.number_of_nodes() # begin of child nodes
    node_inner_begin = 0
    node_inner_end = G.number_of_nodes()

    for level_iter in range(level):

        for node_inner in range(node_inner_begin, node_inner_end):
            if node_inner > node_inner_begin:
                G.add_edge(node_inner, get_pre_node_child(G, node_inner), weight=1)
                if not VERBOSE:
                    print(str(node_inner) + ' ' + str(get_pre_node_child(G, node_inner)))

            if node_inner == node_inner_end - 1 and node_inner != 0:
                G.add_edge(node_inner, node_inner_end, weight=1)
                if not VERBOSE:
                    print(str(node_inner) + ' ' + str(node_inner_end))

            num_node_child = degree - G.degree[node_inner]
            G_node_inner = G.nodes[node_inner]
            radian_node_child = (G_node_inner['radian_end'] - G_node_inner['radian_begin']) / num_node_child
            radian_iter = G_node_inner['radian_begin']

            for node_child in range(node_begin, node_begin + num_node_child):
                node_child_radian = radian_iter + radian_node_child * 0.5
                G.add_node(
                    node_child,
                    x = get_x(node_child_radian, level_iter + 1),
                    y = get_y(node_child_radian, level_iter + 1),
                    radian_begin = radian_iter,
                    radian_end = radian_iter + radian_node_child
                )
                radian_iter = radian_iter + radian_node_child

                G.add_edge(node_inner, node_child, weight=1)
                if not VERBOSE:
                    print(str(node_inner) + ' ' + str(node_child))
            node_begin = node_begin + num_node_child

        node_inner_begin = node_inner_end
        node_inner_end = G.number_of_nodes()

        G.add_edge(node_inner_begin, node_inner_end - 1, weight = 1)
        if not VERBOSE:
            print(str(node_inner_end - 1) + ' ' + str(node_inner_begin))
        for node in range(node_inner_begin + 1, node_inner_end):
            G.add_edge(node - 1, node, weight=1)
            if not VERBOSE:
                print(str(node - 1) + ' ' + str(node))

    return G

def get_pre_node_child(G, node):
    return list(G.edges(node - 1))[-1][1]

def get_x(radian, radius):
    return radius * math.cos(radian)

def get_y(radian, radius):
    return radius * math.sin(radian)

In [3]:
def getNumNodes(level=2):
    res = []
    if level == 0:
        return 1
    else:
        res.append(0)
    if level == 1:
        return 8
    else:
        res.append(7)
    for l in range(level - 1):
        res.append(3*res[-1] - res[-2])
    return sum(res) + 1

In [4]:
for i in range(10):
    print(getNumNodes(i))

1
8
29
85
232
617
1625
4264
11173
29261


In [5]:
G = hyper_gen(3, 7, True)

In [6]:
# utils.plotDeployment(G)

In [7]:
start_time = time.time()
utils.getHyperbolicity(G)
print("--- %s seconds ---" % (time.time() - start_time))

(0, 1, 2, 3)
[2, 2, 3]
(0, 9, 12, 37)
[4, 4, 6]
hyperbolicity: 1.0
--- 2.000075101852417 seconds ---


In [8]:
start_time = time.time()
utils.getHyperbolicity(G)
print("--- %s seconds ---" % (time.time() - start_time))

(0, 1, 2, 3)
[2, 2, 3]
(0, 9, 12, 37)
[4, 4, 6]
hyperbolicity: 1.0
--- 2.066446304321289 seconds ---


In [ ]:
def addEdgesToGraph(G_modified):
    old_edges = list(G_modified.edges)
    i_node = len(G_modified.nodes)
    for edge in old_edges:
        G_modified.remove_edge(edge[0], edge[1])
        G_modified.add_edge(edge[0], i_node, weight = 1)
        G_modified.add_edge(i_node, i_node + 1, weight = 1)
        G_modified.add_edge(i_node + 1, edge[1], weight = 1)
        i_node = i_node + 2
    return G_modified

In [16]:
G37 = hyper_gen(3, 7, True)
G37_modified = addEdgesToGraph(G37)

In [17]:
len(G37_modified.nodes)

477

In [10]:
G27 = hyper_gen(2, 7, True)
G27_modified = addEdgesToGraph(G27)

In [12]:
len(G27_modified.nodes)

155

In [45]:
# To Run, H37 modified?
start_time = time.time()
utils.getHyperbolicity(G_modified)
print("--- %s seconds ---" % (time.time() - start_time))

(0, 1, 2, 3)
[4, 4, 6]
(0, 9, 12, 37)
[8, 8, 12]
hyperbolicity: 2.0
--- 381.9695656299591 seconds ---


In [18]:
start_time = time.time()
utils.getHyperbolicity(G27_modified)
print("--- %s seconds ---" % (time.time() - start_time))

(0, 1, 2, 3)
[6, 6, 9]
(0, 8, 30, 42)
[6, 6, 10]
(0, 9, 12, 119)
[10, 11, 16]
(1, 4, 13, 15)
[12, 12, 18]
hyperbolicity: 3.0
--- 23.71558380126953 seconds ---


## The number of nodes and edges in 10 levels

In [24]:
tmp = []
for level in range(0, 11):
    G = hyper_gen(level, 7, True)
    lst = [level, len(G.nodes), len(G.edges)]
    tmp.append(lst)

### Number of *all* nodes and edges for different level

In [29]:
for lst in tmp:
    print(str(lst[0]) + "\t" + str(lst[1]) + "\t" + str(lst[2]))

0	1	0
1	8	14
2	29	63
3	85	196
4	232	546
5	617	1463
6	1625	3864
7	4264	10150
8	11173	26607
9	29261	69692
10	76616	182490


### Number of nodes and edges at each level

In [27]:
pre_n = 0
pre_e = 0
for lst in tmp:
    print(str(lst[0]) + "\t" + str(lst[1] - pre_n) + "\t" + str(lst[2] - pre_e))
    pre_n = lst[1]
    pre_e = lst[2]

0	1	0
1	7	14
2	21	49
3	56	133
4	147	350
5	385	917
6	1008	2401
7	2639	6286
8	6909	16457
9	18088	43085
10	47355	112798
